<div align="center">

### Plos Articles

</div>


In [1]:
import pandas as pd
import re
# Read the json file 
dataset = pd.read_json("../data/PLOS/PLOSArticles.json")

In [38]:
import re

def extractAbbrev(element):
    try:
        # Regex pattern to find fully uppercase words in parentheses
        pattern = r'\(([A-Z]+)\)'

        # Find all matches with their start positions
        matches = [(match.group(1), match.start()) for match in re.finditer(pattern, element)]
        
        # Function to remove HTML tags from a string
        def remove_html_tags(text):
            return re.sub(r'<[^>]*>', '', text)

        # Extract words based on the new condition
        result = []
        for word, position in matches:
            
            # Find the preceding content up to the word's position
            preceding_text = element[:position].split()
            
            # Extract words starting with the same letter as the abbreviation
            extracted_words = []
            for w in reversed(preceding_text):
                extracted_words.insert(0,w)  # Insert at the beginning of the list
                # Stop extracting if the word doesn't contain the first letter
                if w[0].lower() not in word.lower():
                    break

            result.append((word, [remove_html_tags(item) for item in extracted_words]))  # Remove HTML tags

    except Exception as e:
        print(f"Error: {e}")
        return []
    
    return result
def preprocessData(resultsAnnotation):
    for index in range(len(resultsAnnotation)):
        try :
            word = resultsAnnotation[index][0]
            listOfWords = resultsAnnotation[index][1]
            firstLetter = word[0]
            # Ensure listOfWords has words starting with the same letter as the abbreviation
            while listOfWords and listOfWords[0][0].lower() != firstLetter.lower():
                resultsAnnotation[index][1].pop(0)
        except:
            continue
    return resultsAnnotation